In [ ]:
# Install required libraries
!pip install pandas matplotlib seaborn plotly duckdb openpyxl numpy

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import duckdb as dd
from datetime import datetime
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

All libraries imported successfully!


In [ ]:
# Load data from uploaded Excel file
# 먼저 왼쪽 사이드바에서 problem_data_final.xlsx 파일을 업로드해주세요

Trade = pd.read_excel("problem_data_final.xlsx", sheet_name="Trade")
Funding = pd.read_excel("problem_data_final.xlsx", sheet_name="Funding")
Reward = pd.read_excel("problem_data_final.xlsx", sheet_name="Reward")
Ip = pd.read_excel("problem_data_final.xlsx", sheet_name="IP")
Spec = pd.read_excel("problem_data_final.xlsx", sheet_name="Spec")

print("Data loading completed!")
print(f"Trade shape: {Trade.shape}")
print(f"Funding shape: {Funding.shape}")
print(f"Reward shape: {Reward.shape}")
print(f"IP shape: {Ip.shape}")
print(f"Spec shape: {Spec.shape}")

Data loading completed!
Trade shape: (52953, 10)
Funding shape: (52694, 4)
Reward shape: (198, 3)
IP shape: (2231, 2)
Spec shape: (98207, 4)


In [ ]:
# Data preprocessing
Trade['ts'] = pd.to_datetime(Trade['ts'])
Funding['ts'] = pd.to_datetime(Funding['ts'])
Reward['ts'] = pd.to_datetime(Reward['ts'])

# Add time-based columns for analysis
Trade['minute'] = Trade['ts'].dt.floor('min')
Trade['hour'] = Trade['ts'].dt.floor('H')

# Sort by timestamp
Trade = Trade.sort_values('ts').reset_index(drop=True)
Funding = Funding.sort_values('ts').reset_index(drop=True)
Reward = Reward.sort_values('ts').reset_index(drop=True)

print("Data preprocessing completed!")

Data preprocessing completed!


In [ ]:
# Calculate all organized trading indicators

# 1. IP Sharing Analysis (수정됨)
def calculate_ip_sharing():
    # 공유 IP 사용 계정들 식별
    ip_usage = Ip.groupby('ip')['account_id'].nunique()
    shared_ips = ip_usage[ip_usage > 1].index
    shared_ip_accounts = Ip[Ip['ip'].isin(shared_ips)]['account_id'].unique()

    ip_sharing_data = []
    for account in Ip['account_id'].unique():
        is_sharing = account in shared_ip_accounts
        sharing_ratio = 100.0 if is_sharing else 0.0
        total_ips = len(Ip[Ip['account_id'] == account])

        ip_sharing_data.append({
            'account_id': account,
            'total_ips': total_ips,
            'is_sharing_ip': is_sharing,
            'ip_shared_ratio': sharing_ratio
        })
    return pd.DataFrame(ip_sharing_data)

# 2. Concurrent Trading Analysis
def calculate_concurrent_trading():
    concurrent_data = []
    for account in Trade['account_id'].unique():
        account_trades = Trade[Trade['account_id'] == account]
        account_instances = account_trades.groupby(['minute', 'symbol']).size().reset_index()

        concurrent_count = 0
        total_instances = len(account_instances)

        for _, instance in account_instances.iterrows():
            minute, symbol = instance['minute'], instance['symbol']
            concurrent_accounts = Trade[
                (Trade['minute'] == minute) &
                (Trade['symbol'] == symbol)
            ]['account_id'].nunique()

            if concurrent_accounts > 1:
                concurrent_count += 1

        concurrent_ratio = (concurrent_count / total_instances * 100) if total_instances > 0 else 0

        concurrent_data.append({
            'account_id': account,
            'total_trade_instances': total_instances,
            'concurrent_instances': concurrent_count,
            'concurrent_trading_ratio': concurrent_ratio
        })
    return pd.DataFrame(concurrent_data)

# 3. Price Similarity Analysis
def calculate_price_similarity():
    price_sim_data = []
    for account in Trade['account_id'].unique():
        account_trades = Trade[Trade['account_id'] == account]
        if len(account_trades) < 2:
            continue

        similar_symbols = 0
        total_symbols = 0

        for symbol in account_trades['symbol'].unique():
            symbol_trades = account_trades[account_trades['symbol'] == symbol]
            if len(symbol_trades) > 1:
                price_cv = symbol_trades['price'].std() / symbol_trades['price'].mean()
                if price_cv < 0.01:
                    similar_symbols += 1
                total_symbols += 1

        similarity_ratio = (similar_symbols / total_symbols * 100) if total_symbols > 0 else 0

        price_sim_data.append({
            'account_id': account,
            'total_symbols_traded': total_symbols,
            'similar_price_symbols': similar_symbols,
            'price_similarity_ratio': similarity_ratio
        })
    return pd.DataFrame(price_sim_data)

# 4. Leverage Analysis
def calculate_mean_leverage():
    leverage_data = Trade.groupby('account_id').agg({
        'leverage': ['mean', 'std', 'max', 'min'],
        'amount': ['sum', 'count']
    }).reset_index()

    leverage_data.columns = ['account_id', 'mean_leverage', 'leverage_std', 'max_leverage', 'min_leverage', 'total_amount', 'trade_count']
    return leverage_data

# Calculate all indicators
print("Calculating organized trading indicators...")
ip_sharing_df = calculate_ip_sharing()
concurrent_df = calculate_concurrent_trading()
price_sim_df = calculate_price_similarity()
leverage_df = calculate_mean_leverage()

print("All indicators calculated successfully!")
print(f"IP sharing accounts: {len(ip_sharing_df[ip_sharing_df['ip_shared_ratio'] > 0])}")
print(f"High concurrent trading accounts: {len(concurrent_df[concurrent_df['concurrent_trading_ratio'] > 50])}")
print(f"High price similarity accounts: {len(price_sim_df[price_sim_df['price_similarity_ratio'] > 80])}")
print(f"High leverage accounts: {len(leverage_df[leverage_df['mean_leverage'] > 30])}")

Calculating organized trading indicators...
All indicators calculated successfully!
IP sharing accounts: 17
High concurrent trading accounts: 8
High price similarity accounts: 1
High leverage accounts: 8


In [ ]:
# Graph 1: 두 가지 이상 패턴을 하나의 그래프로 표현 (수정됨)
def plot_dual_suspicious_patterns_v2():
    """다중 IP 사용 + 가장 붐비는 IP 접속자 수 패턴"""

    # IP별 접속 계정 수 계산
    ip_account_counts = Ip.groupby('ip')['account_id'].nunique()

    account_data = []

    for account in Ip['account_id'].unique():
        account_ips = Ip[Ip['account_id'] == account]['ip'].tolist()

        # X축: 총 IP 사용 개수 (다중 IP 패턴)
        total_ips = len(account_ips)

        # Y축: 이 계정이 사용한 IP들 중 가장 붐비는 IP의 접속자 수
        max_crowded_ip = max([ip_account_counts[ip] for ip in account_ips])

        account_data.append({
            'account_id': account,
            'total_ips': total_ips,
            'max_crowded_ip': max_crowded_ip
        })

    account_df = pd.DataFrame(account_data)

    # 산점도 생성
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=account_df['total_ips'],
        y=account_df['max_crowded_ip'],
        mode='markers',
        marker=dict(
            size=6,
            color='red',
            opacity=0.6
        ),
        text=account_df['account_id'],
        hovertemplate='<b>%{text}</b><br>Total IPs: %{x}<br>Max Crowded IP Users: %{y}<extra></extra>'
    ))

    # Trade vs Funding 스타일
    fig.update_layout(
        title="Suspicious IP Usage Patterns",
        xaxis_title="Total IPs Used per Account",
        yaxis_title="Max Users on Shared IP",
        height=500,
        showlegend=False,
        plot_bgcolor='#f0f2f6',
        paper_bgcolor='white'
    )

    fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='#d1d5db', zeroline=False)
    fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='#d1d5db', zeroline=False)

    fig.show()

    return account_df

# 그래프 생성
dual_pattern_data = plot_dual_suspicious_patterns_v2()

print("두 가지 이상 패턴 분석:")
print(f"1. 다중 IP 사용 (X축 > 50): {len(dual_pattern_data[dual_pattern_data['total_ips'] > 50])}개 계정")
print(f"2. 붐비는 IP 사용 (Y축 > 2): {len(dual_pattern_data[dual_pattern_data['max_crowded_ip'] > 2])}개 계정")
print(f"3. 두 패턴 모두: {len(dual_pattern_data[(dual_pattern_data['total_ips'] > 50) & (dual_pattern_data['max_crowded_ip'] > 2)])}개 계정")

두 가지 이상 패턴 분석:
1. 다중 IP 사용 (X축 > 50): 17개 계정
2. 붐비는 IP 사용 (Y축 > 2): 3개 계정
3. 두 패턴 모두: 1개 계정


In [ ]:
# Graph 1: 밝은 배경 + 점 가시성 개선
def plot_density_points_bright_background():
    """밝은 배경에서 모든 점들이 잘 보이게"""

    import numpy as np
    from collections import Counter

    # 데이터 준비
    account_ip_counts = Ip.groupby('account_id')['ip'].nunique()
    ip_account_counts = Ip.groupby('ip')['account_id'].nunique()

    # 서브플롯 생성
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Account IP Usage Density', 'IP Sharing Density'),
        horizontal_spacing=0.15
    )

    # 그래프 1: 계정별
    x_data = list(range(len(account_ip_counts)))
    y_data = account_ip_counts.sort_values().values

    # Y값별 빈도 계산
    y_counts = Counter(y_data)
    density_colors = [y_counts[y] for y in y_data]

    fig.add_trace(
        go.Scatter(
            x=x_data,
            y=y_data,
            mode='markers',
            marker=dict(
                size=8,
                color=density_colors,
                colorscale='Reds',
                showscale=True,
                colorbar=dict(title="Density", x=0.45),
                line=dict(width=2, color='black'),  # 진한 검은 테두리
                cmin=1,  # 최소값 설정해서 너무 하얀색 안 되게
                opacity=0.8
            ),
            text=account_ip_counts.sort_values().index,
            hovertemplate='<b>%{text}</b><br>Index: %{x}<br>IPs: %{y}<br>Density: %{marker.color}<extra></extra>'
        ),
        row=1, col=1
    )

    # 그래프 2: IP별
    x_data2 = list(range(len(ip_account_counts)))
    y_data2 = ip_account_counts.sort_values().values

    # Y값별 빈도 계산
    y_counts2 = Counter(y_data2)
    density_colors2 = [y_counts2[y] for y in y_data2]

    fig.add_trace(
        go.Scatter(
            x=x_data2,
            y=y_data2,
            mode='markers',
            marker=dict(
                size=9,
                color=density_colors2,
                colorscale='Blues',
                showscale=True,
                colorbar=dict(title="Density", x=1.02),
                line=dict(width=2, color='black'),  # 진한 검은 테두리
                cmin=1,  # 최소값 설정
                opacity=0.8
            ),
            text=ip_account_counts.sort_values().index,
            hovertemplate='<b>%{text}</b><br>Index: %{x}<br>Accounts: %{y}<br>Density: %{marker.color}<extra></extra>'
        ),
        row=1, col=2
    )

    # 다시 밝은 배경 (Trade vs Funding 스타일)
    fig.update_layout(
        height=500,
        title_text="Density-Colored Point Analysis",
        showlegend=False,
        plot_bgcolor='#f0f2f6',
        paper_bgcolor='white'
    )

    # 축 설정
    fig.update_xaxes(title_text="Account Index", showgrid=True, gridwidth=0.5, gridcolor='#d1d5db', zeroline=False, row=1, col=1)
    fig.update_yaxes(title_text="IPs Used", showgrid=True, gridwidth=0.5, gridcolor='#d1d5db', zeroline=False, row=1, col=1)
    fig.update_xaxes(title_text="IP Index", showgrid=True, gridwidth=0.5, gridcolor='#d1d5db', zeroline=False, row=1, col=2)
    fig.update_yaxes(title_text="Accounts per IP", showgrid=True, gridwidth=0.5, gridcolor='#d1d5db', zeroline=False, row=1, col=2)

    fig.show()

# 그래프 생성
plot_density_points_bright_background()

In [ ]:
# Graph 2: Concurrent Trading Ratio Distribution
def plot_concurrent_trading_individual():
    fig = go.Figure()

    # Histogram
    fig.add_trace(go.Histogram(
        x=concurrent_df['concurrent_trading_ratio'],
        nbinsx=20,
        name='Concurrent Trading Ratio Distribution',
        marker_color='lightgreen',
        opacity=0.7
    ))

    # Threshold lines
    fig.add_vline(x=30, line_dash="dash", line_color="orange",
                  annotation_text="30% (Suspicious)")
    fig.add_vline(x=50, line_dash="dash", line_color="red",
                  annotation_text="50% (High Risk)")

    fig.update_layout(
        title="Concurrent Trading Ratio Distribution",
        xaxis_title="Concurrent Trading Ratio (%)",
        yaxis_title="Account Count",
        height=500,
        showlegend=False
    )

    fig.show()

plot_concurrent_trading_individual()
print(f"동시 거래 분석 - 평균: {concurrent_df['concurrent_trading_ratio'].mean():.2f}%, 95분위수: {concurrent_df['concurrent_trading_ratio'].quantile(0.95):.2f}%")
print(f"50% 이상 동시 거래: {len(concurrent_df[concurrent_df['concurrent_trading_ratio'] >= 50])}개 계정")

동시 거래 분석 - 평균: 18.47%, 95분위수: 71.59%
50% 이상 동시 거래: 8개 계정


In [ ]:
# 50% 이상 동시 거래 계정들 상세 확인
def check_high_concurrent_accounts():
    """50% 이상 동시 거래하는 계정들을 구체적으로 확인"""

    # 기존 동시 거래 데이터 다시 계산
    concurrent_data = []

    for account in Trade['account_id'].unique():
        account_trades = Trade[Trade['account_id'] == account]
        account_instances = account_trades.groupby(['minute', 'symbol']).size().reset_index()

        concurrent_count = 0
        total_instances = len(account_instances)

        for _, instance in account_instances.iterrows():
            minute, symbol = instance['minute'], instance['symbol']
            concurrent_accounts = Trade[
                (Trade['minute'] == minute) &
                (Trade['symbol'] == symbol)
            ]['account_id'].nunique()

            if concurrent_accounts > 1:
                concurrent_count += 1

        concurrent_ratio = (concurrent_count / total_instances * 100) if total_instances > 0 else 0

        concurrent_data.append({
            'account_id': account,
            'total_instances': total_instances,
            'concurrent_instances': concurrent_count,
            'concurrent_ratio': concurrent_ratio
        })

    concurrent_df = pd.DataFrame(concurrent_data)

    # 50% 이상 동시 거래 계정들 필터링
    high_concurrent = concurrent_df[concurrent_df['concurrent_ratio'] >= 50].sort_values('concurrent_ratio', ascending=False)

    print(f"=== 50% 이상 동시 거래 계정 분석 ===")
    print(f"총 계정 수: {len(concurrent_df)}")
    print(f"50% 이상 동시 거래 계정: {len(high_concurrent)}개")
    print(f"전체 평균 동시 거래 비율: {concurrent_df['concurrent_ratio'].mean():.2f}%")
    print(f"95분위수: {concurrent_df['concurrent_ratio'].quantile(0.95):.2f}%")

    if len(high_concurrent) > 0:
        print(f"\n=== 50% 이상 동시 거래 계정 상세 목록 ===")
        for i, (_, row) in enumerate(high_concurrent.iterrows(), 1):
            print(f"{i:2d}. {row['account_id']}: {row['concurrent_ratio']:.1f}% ({row['concurrent_instances']}/{row['total_instances']})")

    # 구간별 분포도 확인
    print(f"\n=== 동시 거래 비율 구간별 분포 ===")
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    for i in range(len(bins)-1):
        start, end = bins[i], bins[i+1]
        count = len(concurrent_df[(concurrent_df['concurrent_ratio'] >= start) & (concurrent_df['concurrent_ratio'] < end)])
        print(f"{start:2d}%-{end:2d}%: {count:2d}개 계정")

    # 100% 동시 거래 계정 (특별히 확인)
    perfect_concurrent = high_concurrent[high_concurrent['concurrent_ratio'] == 100]
    if len(perfect_concurrent) > 0:
        print(f"\n=== 100% 동시 거래 계정 (의심도 극대) ===")
        for _, row in perfect_concurrent.iterrows():
            print(f"- {row['account_id']}: 모든 거래가 동시 거래 ({row['total_instances']}건 중 {row['concurrent_instances']}건)")

    return high_concurrent

# 실행
high_concurrent_accounts = check_high_concurrent_accounts()

=== 50% 이상 동시 거래 계정 분석 ===
총 계정 수: 63
50% 이상 동시 거래 계정: 8개
전체 평균 동시 거래 비율: 18.47%
95분위수: 71.59%

=== 50% 이상 동시 거래 계정 상세 목록 ===
 1. A_d444580218: 96.7% (381/394)
 2. A_55021b4ae2: 82.4% (28/34)
 3. A_6b3a8b213d: 81.5% (22/27)
 4. A_c91db6cabf: 71.8% (28/39)
 5. A_e95b2c5030: 69.7% (23/33)
 6. A_8413984c30: 69.0% (80/116)
 7. A_8860be39b3: 67.8% (80/118)
 8. A_f96ede8d34: 56.1% (384/684)

=== 동시 거래 비율 구간별 분포 ===
 0%-10%: 35개 계정
10%-20%: 11개 계정
20%-30%:  7개 계정
30%-40%:  1개 계정
40%-50%:  1개 계정
50%-60%:  1개 계정
60%-70%:  3개 계정
70%-80%:  1개 계정
80%-90%:  2개 계정
90%-100%:  1개 계정


In [ ]:
# 동시 거래 분석 데이터 추출 및 검증
def extract_concurrent_trading_data():
    """동시 거래 관련 모든 데이터를 추출해서 직접 검증"""

    print("=== 동시 거래 분석 데이터 추출 ===")

    # 1. 기본 통계
    print(f"전체 거래 건수: {len(Trade):,}")
    print(f"전체 계정 수: {Trade['account_id'].nunique()}")
    print(f"전체 심볼 수: {Trade['symbol'].nunique()}")
    print(f"거래 기간: {Trade['ts'].min()} ~ {Trade['ts'].max()}")

    # 2. 샘플 계정의 상세 데이터
    sample_accounts = Trade['account_id'].unique()[:3]

    for account in sample_accounts:
        print(f"\n--- {account} 상세 분석 ---")
        account_trades = Trade[Trade['account_id'] == account]

        # 분 단위 그룹핑
        minute_groups = account_trades.groupby(['minute', 'symbol']).size().reset_index()
        minute_groups.columns = ['minute', 'symbol', 'trade_count']

        print(f"총 거래 인스턴스 (분+심볼): {len(minute_groups)}개")
        print(f"거래한 심볼: {account_trades['symbol'].nunique()}개")

        # 동시 거래 확인
        concurrent_instances = []

        for _, instance in minute_groups.head(10).iterrows():  # 처음 10개만
            minute, symbol = instance['minute'], instance['symbol']

            # 같은 시간+심볼에 거래한 다른 계정들
            concurrent_accounts = Trade[
                (Trade['minute'] == minute) &
                (Trade['symbol'] == symbol)
            ]['account_id'].unique()

            is_concurrent = len(concurrent_accounts) > 1

            concurrent_instances.append({
                'minute': minute,
                'symbol': symbol,
                'concurrent_accounts': len(concurrent_accounts),
                'is_concurrent': is_concurrent,
                'other_accounts': [acc for acc in concurrent_accounts if acc != account]
            })

            print(f"  {minute} {symbol}: {len(concurrent_accounts)}개 계정 {'(동시!)' if is_concurrent else '(단독)'}")
            if is_concurrent:
                print(f"    함께 거래한 계정: {[acc for acc in concurrent_accounts if acc != account]}")

    # 3. 전체 동시 거래 통계
    print(f"\n=== 전체 동시 거래 패턴 ===")

    # 분+심볼별 계정 수 분포
    minute_symbol_stats = Trade.groupby(['minute', 'symbol'])['account_id'].nunique().reset_index()
    minute_symbol_stats.columns = ['minute', 'symbol', 'account_count']

    print(f"총 거래 인스턴스 (분+심볼): {len(minute_symbol_stats):,}개")
    print(f"1개 계정만 거래: {len(minute_symbol_stats[minute_symbol_stats['account_count'] == 1]):,}개")
    print(f"2개 이상 계정 동시 거래: {len(minute_symbol_stats[minute_symbol_stats['account_count'] >= 2]):,}개")
    print(f"최대 동시 거래 계정 수: {minute_symbol_stats['account_count'].max()}개")

    # 동시 거래가 많은 시점들
    high_concurrent = minute_symbol_stats[minute_symbol_stats['account_count'] >= 3].sort_values('account_count', ascending=False)
    if len(high_concurrent) > 0:
        print(f"\n=== 높은 동시 거래 시점들 (Top 5) ===")
        for _, row in high_concurrent.head().iterrows():
            print(f"{row['minute']} {row['symbol']}: {row['account_count']}개 계정")

    return minute_symbol_stats

# 데이터 추출 실행
concurrent_stats = extract_concurrent_trading_data()

=== 동시 거래 분석 데이터 추출 ===
전체 거래 건수: 52,953
전체 계정 수: 63
전체 심볼 수: 401
거래 기간: 2025-03-01 00:04:14.329767 ~ 2025-10-31 20:00:38.588185

--- A_c668740478 상세 분석 ---
총 거래 인스턴스 (분+심볼): 972개
거래한 심볼: 3개
  2025-03-01 00:04:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 00:09:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 00:30:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 00:34:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 00:44:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 01:43:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 01:50:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 02:44:00 BTCUSDT.PERP: 2개 계정 (동시!)
    함께 거래한 계정: ['A_0bfb0b4311']
  2025-03-01 02:47:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 02:53:00 BTCUSDT.PERP: 2개 계정 (동시!)
    함께 거래한 계정: ['A_cab220ef3c']

--- A_7e27164ab6 상세 분석 ---
총 거래 인스턴스 (분+심볼): 786개
거래한 심볼: 5개
  2025-03-01 00:23:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 01:02:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 03:53:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-01 04:58:00 BTCUSDT.PERP: 1개 계정 (단독)
  2025-03-02 03:28:00 B

In [ ]:
# Graph 3: Price Similarity Ratio Distribution
def plot_price_similarity_individual():
    fig = go.Figure()

    # Histogram
    fig.add_trace(go.Histogram(
        x=price_sim_df['price_similarity_ratio'],
        nbinsx=20,
        name='Price Similarity Ratio Distribution',
        marker_color='lightsalmon',
        opacity=0.7
    ))

    # Threshold lines
    fig.add_vline(x=60, line_dash="dash", line_color="orange",
                  annotation_text="60% (Suspicious)")
    fig.add_vline(x=80, line_dash="dash", line_color="red",
                  annotation_text="80% (High Risk)")

    fig.update_layout(
        title="Price Similarity Ratio Distribution",
        xaxis_title="Price Similarity Ratio (%)",
        yaxis_title="Account Count",
        height=500,
        showlegend=False
    )

    fig.show()

plot_price_similarity_individual()
valid_accounts = len(price_sim_df[price_sim_df['total_symbols_traded'] > 0])
print(f"가격 유사도 분석 - 평균: {price_sim_df['price_similarity_ratio'].mean():.2f}%")
print(f"유사 거래가 있는 계정: {valid_accounts}개")
print(f"80% 이상 유사: {len(price_sim_df[price_sim_df['price_similarity_ratio'] >= 80])}개 계정")

가격 유사도 분석 - 평균: 29.06%
유사 거래가 있는 계정: 63개
80% 이상 유사: 1개 계정


In [ ]:
# Graph 4: Mean Leverage Distribution
def plot_mean_leverage_individual():
    fig = go.Figure()

    # Histogram
    fig.add_trace(go.Histogram(
        x=leverage_df['mean_leverage'],
        nbinsx=20,
        name='Mean Leverage Distribution',
        marker_color='lightcoral',
        opacity=0.7
    ))

    # Threshold lines
    fig.add_vline(x=30, line_dash="dash", line_color="orange",
                  annotation_text="30x (Suspicious)")
    fig.add_vline(x=50, line_dash="dash", line_color="red",
                  annotation_text="50x (High Risk)")

    fig.update_layout(
        title="Mean Leverage Distribution",
        xaxis_title="Mean Leverage (x)",
        yaxis_title="Account Count",
        height=500,
        showlegend=False
    )

    fig.show()

plot_mean_leverage_individual()
print(f"레버리지 분석 - 평균: {leverage_df['mean_leverage'].mean():.2f}x, 중앙값: {leverage_df['mean_leverage'].median():.2f}x")
print(f"30x 이상: {len(leverage_df[leverage_df['mean_leverage'] > 30])}개 계정")
print(f"50x 이상: {len(leverage_df[leverage_df['mean_leverage'] > 50])}개 계정")

레버리지 분석 - 평균: 12.42x, 중앙값: 7.86x
30x 이상: 8개 계정
50x 이상: 2개 계정


In [ ]:
# 수정된 다중 IP 사용 그래프
def plot_multiple_ip_usage():
    # 계정별 IP 사용 개수 계산
    ip_counts_per_account = Ip.groupby('account_id')['ip'].nunique().reset_index()
    ip_counts_per_account.columns = ['account_id', 'ip_count']

    fig = go.Figure()

    # 히스토그램
    fig.add_trace(go.Histogram(
        x=ip_counts_per_account['ip_count'],
        nbinsx=20,
        name='Multiple IP Usage Distribution',
        marker_color='lightpink',
        opacity=0.7
    ))

    # 임계값 라인
    fig.add_vline(x=10, line_dash="dash", line_color="orange",
                  annotation_text="10+ IPs (Suspicious)")
    fig.add_vline(x=50, line_dash="dash", line_color="red",
                  annotation_text="50+ IPs (High Risk)")
    fig.add_vline(x=100, line_dash="dash", line_color="darkred",
                  annotation_text="100+ IPs (Extreme Risk)")

    fig.update_layout(
        title="Multiple IP Usage Distribution",
        xaxis_title="Number of IPs Used per Account",
        yaxis_title="Account Count",
        height=500,
        showlegend=False
    )

    fig.show()

plot_multiple_ip_usage()

# 통계 정보
ip_counts_per_account = Ip.groupby('account_id')['ip'].nunique()
print(f"다중 IP 사용 분석:")
print(f"평균 IP 사용 개수: {ip_counts_per_account.mean():.2f}개")
print(f"중앙값 IP 사용: {ip_counts_per_account.median():.0f}개")
print(f"10개 이상 IP 사용: {len(ip_counts_per_account[ip_counts_per_account >= 10])}개 계정")
print(f"50개 이상 IP 사용: {len(ip_counts_per_account[ip_counts_per_account >= 50])}개 계정")
print(f"100개 이상 IP 사용: {len(ip_counts_per_account[ip_counts_per_account >= 100])}개 계정")
print(f"최대 IP 사용: {ip_counts_per_account.max()}개")

# 극단적 사용자들
extreme_users = ip_counts_per_account[ip_counts_per_account >= 100]
print(f"\n극단적 IP 사용 계정들:")
for account, count in extreme_users.head().items():
    print(f"  {account}: {count}개 IP")

다중 IP 사용 분석:
평균 IP 사용 개수: 35.41개
중앙값 IP 사용: 24개
10개 이상 IP 사용: 52개 계정
50개 이상 IP 사용: 17개 계정
100개 이상 IP 사용: 1개 계정
최대 IP 사용: 177개

극단적 IP 사용 계정들:
  A_ebdb869fd3: 177개 IP
